In [161]:
import pandas as pd
import numpy as np
import string

wordlen = 5

def calWordScore(allowed):
        frequency = { ch : [0] * 5 for ch in string.ascii_lowercase}
        for word in allowed:
            for i in range(0,5):
                frequency[word[i]][i] += 1
        
        matrix  = pd.DataFrame(frequency).T
        matrix  = (matrix-matrix.min())/(matrix.max()-matrix.min())
        matrix = matrix.T.to_dict()

        wordScore = {}
        for word in allowed:
            score = 0
            for i in range(0,5):
                score += matrix[word[i]][i]
            wordScore.update({word: round(score,3)})
        #print(wordScore)
        return wordScore


In [162]:
wf = pd.read_csv('unigram_freq.csv')
wf['len'] = wf["word"].str.len()
wf = wf[wf['len'] == wordlen]
wf = wf[['word','count']]
wf['prob'] = (wf['count']- wf['count'].min())/( wf['count'].max()- wf['count'].min())

allowed = pd.read_csv('12972_allowed_words_entroy.txt',delimiter=" ",names=['word','entropy'])
wordproba  = wf.merge(allowed,how='right')
wordproba = wordproba.fillna(0.0)

wordproba = wordproba[['word','entropy','prob']]
pf = pd.DataFrame(calWordScore(allowed.word.values),index = ["posFreq"]).T
wordproba = wordproba.merge(pf,how='right',right_index= True,left_on="word")
wordproba


,word,entropy,prob,posFreq
0,aahed,4.291696,0.000000,2.760
1,aalii,3.701130,0.000000,2.596
2,aargh,3.979746,0.000048,2.708
3,aarti,4.619455,0.000041,2.890
4,abaca,2.991318,0.000055,1.844
...,...,...,...,...
12967,zuzim,2.830609,0.000000,1.108
12968,zygal,3.730280,0.000000,1.041
12969,zygon,3.502955,0.000002,0.893
12970,zymes,4.399831,0.000009,2.580


In [163]:
wordproba.describe()

,entropy,prob,posFreq
count,12972.000000,12972.000000,12972.000000
mean,4.540141,0.003969,2.521574
std,0.627250,0.025490,0.637885
min,1.900629,0.000000,0.530000
25%,4.143419,0.000000,2.066000
50%,4.588342,0.000024,2.494500
75%,4.990755,0.000368,2.961000
max,6.194053,1.000000,4.895000


In [164]:
wordproba.loc[wordproba['prob'] > 0.003969,"prob"] = 0.003969

In [177]:
wordproba['prob'] = (wordproba['entropy'] + wordproba['posFreq'])**2 * wordproba['prob'] 
wordproba['posEntropy'] = (wordproba['entropy']  + wordproba['posFreq'] )/2
wordproba

,word,entropy,prob,posFreq,posEntropy
0,aahed,4.291696,0.000000e+00,2.760,3.525848
1,aalii,3.701130,0.000000e+00,2.596,3.148565
2,aargh,3.979746,3.842260e+05,2.708,3.343873
3,aarti,4.619455,1.325499e+06,2.890,3.754728
4,abaca,2.991318,8.938644e+03,1.844,2.417659
...,...,...,...,...,...
12967,zuzim,2.830609,0.000000e+00,1.108,1.969305
12968,zygal,3.730280,0.000000e+00,1.041,2.385640
12969,zygon,3.502955,1.099068e+02,0.893,2.197977
12970,zymes,4.399831,1.245337e+05,2.580,3.489915


In [178]:
wordproba.describe()

,entropy,prob,posFreq,posEntropy
count,12972.000000,1.297200e+04,12972.000000,12972.000000
mean,4.540141,3.897674e+07,2.521574,3.530858
std,0.627250,2.041903e+08,0.637885,0.553805
min,1.900629,0.000000e+00,0.530000,1.501756
25%,4.143419,0.000000e+00,2.066000,3.150477
50%,4.588342,2.031475e+05,2.494500,3.539958
75%,4.990755,6.506720e+06,2.961000,3.909796
max,6.194053,7.109515e+09,4.895000,5.339526


In [142]:
#wordproba.set_index('word',inplace= True)
#wordproba.loc['ratch']

In [166]:
wordproba1 = wordproba[['word','prob']]

In [167]:
wordproba1

,word,prob
0,aahed,0.000000
1,aalii,0.000000
2,aargh,0.002147
3,aarti,0.002324
4,abaca,0.001279
...,...,...
12967,zuzim,0.000000
12968,zygal,0.000000
12969,zygon,0.000041
12970,zymes,0.000454


In [169]:
wordproba1.to_csv('word_frequency.csv',index=False)

In [180]:
wordproba2 = wordproba[['word','posFreq']]
wordproba2.to_csv('12972_allowed_words_entroy.txt',index=False)